## Backtest Pipeline Example
In this example, we will run a full backtest pipeline for a trading strategy.

### Steps:

1. Download a insample dataset to detect backtest date ranges
2. Run strategy over each range and save results
3. Show results in a backtest report

## Setup constants for backtesting

In [1]:
# Constants
STRATEGY_NAME = 'StrategyV1'
DATA_STORAGE_PATH = "resources/backtest_data"
BACKTEST_RESULTS_STORAGE_PATH = "resources/backtest_results"
BACKTEST_PERMUTATION_RESULTS_STORAGE_PATH = "resources/backtest_permutation_results"

# Asset constants
INSAMPLE_ASSETS = ["BTC/EUR", "ETH/EUR"]
OUTSAMPLE_ASSETS = ["XRP/EUR", "DOT/EUR"]

# Time range constants
INSAMPLE_START_DATE = "2020-01-01"
INSAMPLE_END_DATE = "2024-01-01"
OUTSAMPLE_START_DATE = "2024-01-01"
OUTSAMPLE_END_DATE = "2025-01-01"

## Dynamically loadin the strategy
We can dynamically load the strategy from a file. This is useful if we want to change the strategy without restarting the entire
notebook. This allows you to edit the strategy and quickly test it.

In [3]:
import importlib
import strategies.strategy_v1 as strategy_module_v1  # Import the module

importlib.reload(strategy_module_v1)  # Reload the module
configure_strategy = strategy_module_v1.configure_strategy  # Re-bind the class

## Select in sample test date ranges

Date ranges for in-sample testing should be atleast 2 years long so we can calculate 
all metrics. Three date ranges are selected, coresponding to an uptrend, a downtrend and a sideways trend. Using these three date ranges we can test the performance of the strategy in different market conditions.

We will use a window=730, which means 2-year slices of market behavior that we can use to detect uptrends, downtrends and sideways trends.


In [3]:
from investing_algorithm_framework import download

data = download(
    "BTC/EUR",
    time_frame="1h",
    start_date="2020-01-01",
    end_date="2025-6-30",
    window_size=200,
    pandas=True,
    market="bitvavo",
    storage_path="resources/backtest_data",
)

Error reading data from resources/backtest_data/BTC_EUR_bitvavo_1h_2020010100_2025063000.csv: ExprStringNameSpace.strptime() got an unexpected keyword argument 'fmt'


In [4]:
from investing_algorithm_framework import select_backtest_date_ranges

# Select date ranges for backtesting with a window of 730 days (2 years), so
# we have clear uptrend, downtrend, and sideways market conditions.
ranges = select_backtest_date_ranges(data, window=730)

for r in ranges:
    print(r)

UpTurn: 2020-03-13 01:00:00 - 2022-03-13 01:00:00
DownTurn: 2021-03-10 17:00:00 - 2023-03-10 17:00:00
SideWays: 2022-01-05 19:00:00 - 2024-01-05 19:00:00


## Run in-sample backtest
We will run the strategy over each date range and save the results. This allows us to analyze the performance of the strategy in different market conditions. We will save the results to a file so we can analyze them later

In [ ]:
from datetime import datetime
from investing_algorithm_framework import SnapshotInterval, create_app, \
    TimeUnit, TimeFrame
from investing_algorithm_framework import create_app, BacktestDateRange

app = create_app()


# Configure the strategy with the insample assets. This will create the
# strategy instance configured with the right data providers.
strategy = configure_strategy(
    time_unit=TimeUnit.HOUR,
    interval=2,
    time_frame=TimeFrame.TWO_HOUR,
    fast=50,
    slow=100,
    trend=200,
    stop_loss_percentage=2,
    stop_loss_sell_size=50,
    take_profit_percentage=8,
    take_profit_sell_size=50,
    assets=INSAMPLE_ASSETS
)
app.add_strategy(strategy)
app.add_market(market="BITVAVO", trading_symbol="EUR")

# ranges = ranges[:1]
start_date = datetime(2021, 1, 1)
end_date = datetime(2021, 3, 3)
ranges = [
    BacktestDateRange(
        name="Insample",
        start_date=start_date,
        end_date=end_date
    )
]

for range in ranges:
    print(f"\nBacktest {range}")
    name = f"{STRATEGY_NAME}_{range.name.replace(' ', '_')}"
    result = app.run_backtest(
        name=name,
        backtest_date_range=range,
        initial_amount=1000,
        snapshot_interval=SnapshotInterval.STRATEGY_ITERATION,
        risk_free_rate=2.4,
        save=False
    )


NameError: name 'configure_strategy' is not defined

## Show in sample backtest results


In [2]:
from investing_algorithm_framework import BacktestReport

report = BacktestReport.open(
    backtests=[result]
)
report.show(browser=True)

NameError: name 'result' is not defined